### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
mushroomの50サンプル分

### yのサンプリング
- コンパイルを行うまでは`pyqubo`
- SAは`SimulatedAnnealingSampler()`

### 改訂(08/03)
- ori_tiのkeyがなかった場合に1とする必要がない
- sum([j[1] for j in ti_list])は必ず100である

### 改訂（08/04）
- samplingでnum=100とする。


### 改訂（08/18）
- mushroom_lの1の総数を8にしよう。(今までは3だった)
- A4を2枚
- 幅を付けた図を描く
- tを0, 1と増やす

In [44]:
#!pip3 install dwave-neal

In [45]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import sys

In [46]:
X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0) #50bit
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [47]:
num_samples
len(columns)

118

In [48]:
num_cols

118

In [49]:
y_sum

8

## t^0だけでサンプリングをしてみる

- minPの特徴量を知る（10回とも同じ？、）

In [50]:
#numreadsは作るサンプル数（ダブリはあるはず）
#numreadsは100, 500, 1000とするか、logをとるか(np.logspace)

numreads_10 = np.logspace(1, 4, num=4, base=10, dtype=int)
len(numreads_10) #4
print(numreads_10)

numreads_5 = [ll*100 for ll in range(1, 21) if ll%5==0 or ll==1]
len(numreads_5) #5
print(numreads_5)

[   10   100  1000 10000]
[100, 500, 1000, 1500, 2000]


In [35]:
## 図の表示用セル

%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [100, 200]  # 図の縦横のサイズ([横(inch),縦(inch)])
plt.rcParams["figure.autolayout"] = True   # レイアウトの自動調整を利用するかどうか
plt.rcParams["figure.subplot.wspace"] = 0.10# 図が複数枚ある時の左右との余白
plt.rcParams["figure.subplot.hspace"] = 0.10# 図が複数枚ある時の上下との余白

fig, ax = plt.subplots(int(num_cols/3)+1, 3)
#fig, ax = plt.subplots(int(num_cols/2), 2, figsize=(100,100))

y = Array.create('y', shape=num_samples, vartype='BINARY')

#QUBO式で定式化
H = (sum(y) - y_sum)**2
qb = H.compile().to_qubo()
sa_sampler = SimulatedAnnealingSampler()
res = sa_sampler.sample_qubo(qb[0], num_reads=numreads_10[3]) #0, 1, 2, 3
#res = sa_sampler.sample_qubo(qb[0], num_reads=numreads_5[4]) #0, 1, 2, 3, 4

qb_minP_cols = []
servey_num_col_list = columns
qbl_p = {} ##

for index, i in enumerate(servey_num_col_list):
    ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

    ti_dic = {} #{内積の値:該当数}という辞書。

    valid_y_num = 0
    for y_info in list(res.record):
        if sum(y_info[0]) - y_sum == 0: #サンプリングした結果が本当に条件に適しているのか確認
            valid_y_num += 1
            y = pd.Series(y_info[0])

            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1

    ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
    franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
    qbl_p[i] = franc/valid_y_num
    
    '''n_row = int(index%int(num_cols/2)) #行
    
    #列
    if index < int(num_cols/2):
        n_col = 0
    else:
        n_col = 1'''
    
    n_row = int(index%(int(num_cols/3)+1))#行
    
    #列
    if index < int(num_cols/3)+1:
        n_col = 0
    elif index < (int(num_cols/3)+1)*2:
        n_col = 1
    else:
        n_col = 2
    
    #ヒストグラムを出力
    ax[n_row, n_col].set_title(i, fontsize=15)
    ax[n_row, n_col].bar([j[0] for j in ti_list], [j[1] for j in ti_list])
    ax[n_row, n_col].scatter(ori_ti, 100, s=200, marker='D')
    ax[n_row, n_col].set_xlabel('inner product size', fontsize=10)
    ax[n_row, n_col].set_ylabel('number', rotation=90, fontsize=10, labelpad=20)
    
#図が重ならないようにする
fig.tight_layout()

#全体に一つの題名を表示
fig.suptitle('inner product size and its number using sampling y which meet conditions', fontsize=15)

fig.subplots_adjust(bottom=0, left=0, top=0.1, right=0.1, wspace=0)

#図を表示
#plt.show()

#fig.savefig('../output/mushrooml/neal_1000.png')

KeyboardInterrupt: 

In [6]:
def make_y(minP_cols, numreads):
    # BINARY変数
    y = Array.create('y', shape=num_samples, vartype='BINARY')
    
    # t0の条件
    H = (sum(y) - y_sum)**2
    
    # tのそれ以外の条件
    if len(minP_cols) > 0:
        for i in minP_cols:
            H_plus = (np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom))**2
            H = H + H_plus
    
    qb = H.compile().to_qubo()
    sa_sampler = SimulatedAnnealingSampler()

    res = sa_sampler.sample_qubo(qb[0], num_reads=numreads)
    
    # 本当に条件に適しているか確認して新たなリストに格納
    valid_y_list = []
    for y_info in list(res.record):
        if sum(y_info[0]) - y_sum !=0:
            continue
        if len(minP_cols) == 0:
            valid_y_list.append(pd.Series(y_info[0]))
        else:
            for i in minP_cols:
                if np.dot(X_mushroom[i], pd.Series(y_info[0])) - np.dot(X_mushroom[i], y_mushroom)==0:
                    valid_y_list.append(pd.Series(y_info[0]))
                    
    return valid_y_list

## t^0, t^1だけ

- 平均などt^0のときと同じ図を出力する
- 一番小さいiを知る


In [ ]:
minP_cols = []
servey_num_col_list = columns

for _ in [0, 1]
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

        ti_dic = {} #{内積の値:該当数}という辞書。

        valid_y_list = make_y(minP_cols, numreads)

        for y in valid_y_list:      
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1

        ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
        franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
        qbl_p[i] = franc/len(valid_y_list)


    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]

## t（条件）をすべて使ってみる

In [ ]:
minP_cols = []
servey_num_col_list = columns

while len(servey_num_col_list) > 0:    
    p = {}
    
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        
        valid_y_list = make_y(minP_cols, numreads)
        
        for y in valid_y_list:      
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        count_better_t = sum([v for k, v in ti_dic.items() if k >= ori_ti])
        p[i] = count_better_t / distri_element_num

        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]

In [7]:
minP_cols = []
servey_num_col_list = columns
distri_element_num = 100

while len(servey_num_col_list) > 0:    
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        
        for _ in range(distri_element_num): 
            y = make_y(minP_cols)
            
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        count_better_t = sum([v for k, v in ti_dic.items() if k >= ori_ti])
        p[i] = count_better_t / distri_element_num

        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]